In [1]:
import pandas as pd
from tqdm import tqdm
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF

In [2]:
cx = sqlite3.connect(r'D:\Python\获取网络信息用于文本分析\data.db')

In [3]:
df2 = pd.read_sql_query('select * from junshi',cx)
df1 = pd.read_sql_query('select * from NBA',cx)
df0 = pd.read_sql_query('select * from gaokao',cx)

In [4]:
df0 = df0.dropna(axis=1, how='any')
df1 = df1.dropna(axis=1, how='any')
df2 = df2.dropna(axis=1, how='any')

In [5]:
junshi = df2.to_dict(orient='records') 
nba = df1.to_dict(orient='records')
gaokao = df0.to_dict(orient='records')

In [6]:
lable = []
for i in range(len(junshi[3])-1):
    lable.append(0)
for i in range(len(junshi[3])-1,len(junshi[3])-1+len(gaokao[3])-1):
    lable.append(1)
for i in range(len(junshi[3])-1+len(gaokao[3])-1,len(junshi[3])-1+len(gaokao[3])-1+len(nba[3])-1):
    lable.append(2)
len(lable)

1122

In [7]:
data1 = []
for w in junshi[3]:
        data1.append(junshi[3][w])
for w in gaokao[3]:
        data1.append(gaokao[3][w])
for w in nba[3]:
        data1.append(nba[3][w])

In [8]:
data1.pop(0)
data1.pop(198)
data1.pop(498)

'acticle'

In [9]:
import re
li = []
for i in range(len(data1)):
    reg = "[^\u4e00-\u9fa5]"
    li.append(re.sub(reg,'',data1[i]))

In [10]:
import jieba
for  w  in range(len(li)):
    data1[w]=jieba.lcut(li[w])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\14982\AppData\Local\Temp\jieba.cache
Loading model cost 0.974 seconds.
Prefix dict has been built successfully.


In [11]:
X_train = []
for i in  range(len(data1)):
    X_train.append(' '.join(data1[i]))
len(X_train)

1122

In [12]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X_train,lable,test_size=0.3,random_state=420)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF
from sklearn.metrics import classification_report
tfidf = TFIDF().fit(Xtrain)
Xtrain_ = tfidf.transform(Xtrain)
Ytrain_ = tfidf.transform(Xtest)


In [14]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB

from sklearn.metrics import brier_score_loss as B
clf = MultinomialNB()
clf.fit(Xtrain_,Ytrain)
y_pred = clf.predict(Ytrain_)
proba = clf.predict_proba(Ytrain_)
score = clf.score(Ytrain_,Ytest)
predict = clf.predict(Ytrain_)

交叉验证

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import TruncatedSVD as SVD
tfidf = TFIDF().fit(X_train)
X_tr = tfidf.transform(X_train)
pca = SVD(n_components=2)           #实例化
pca = pca.fit(X_tr)                    #拟合模型
X_dr = pca.transform(X_tr)             #获取新矩阵


In [16]:
rfc_s = cross_val_score( GaussianNB(),X_dr,lable,cv=10)
rfc_s.mean()

0.9436757554014191

In [17]:
print(score)

0.9762611275964391


In [18]:
print("准确率为：", clf.score(Ytrain_,Ytest))

准确率为： 0.9762611275964391


In [21]:
list1=["军事",'高考','NBA']
print(classification_report(Ytest, predict, target_names=list1))  #精确率（查的准）  召回率（查的全） 二者均值 标签出现的次数

             precision    recall  f1-score   support

         军事       1.00      0.87      0.93        53
         高考       1.00      0.99      0.99        96
        NBA       0.96      1.00      0.98       188

avg / total       0.98      0.98      0.98       337



In [22]:
fr = open('junshi.txt',"r")
st=fr.read()
fr.close()   #将文件关闭

In [23]:
str1 = jieba.lcut(st)

In [24]:
stopwords = [line.strip() for line in open('D:\Python\Python\stopwords\哈工大停用词表.txt',encoding='UTF-8').readlines()]
content = []
line = []
for word in str1:
    if word  not in stopwords:
             line.append(word) 
#content.append(line)
line = ' '.join(line)
line = [line]
line[0]

'英国 近代 历史 中 占据 地位 无人 敢 小觑 遍布 世界各地 殖民地 区 更是 无人 正是 种种原因 才 日不落 称号 一战 二战时期 英国 国力 有所 滑坡 科技 实力 仍 全球 顶尖 当时 喷气式 发动机 正是 英国 带领 下 逐渐 得到 发展 普及 由此来看 英国 战斗机 应该 挑不出 毛病 说 英国 战机 外貌 设计 就要 打个 问号 全球 最丑 战斗机 两个 发动机 摞 一起   活像 一只 大肚 蝈蝈 \n \n 英国 战机 当今世界 军火 市场 上 占据 一席之地 英国 战机 作战 实力 不 含有 水分 看到 英国 战斗机 人 第一 反应 都 感到 会 不 协调 称做 丑 英国 战机 大腹便便 不说 更是 没有 丝毫 优雅 可言 英国 绅士 特征 或者说 气质 有着 本质 不同 好事者 全球 战机 做 排名榜 考核 因素 便是 最丑 榜单 中有 一半 都 英国 战机'

In [25]:
Xte=tfidf.transform(line)

In [26]:

proba =clf.predict_proba(Xte)
y_pred = clf.predict(Xte)
y_pred

ValueError: dimension mismatch

In [27]:
for i in range(3):
    y='{:.3f}'.format(proba[0][i])
    print('你输入的文章有'+y+'的概率属于'+list1[i])

你输入的文章有0.699的概率属于军事
你输入的文章有0.075的概率属于高考
你输入的文章有0.225的概率属于NBA
